# **PROJET DE MASTER 1**

## **OBjectif principal: Analyse de données de criminalité; étude de cas sur les données de la base Crime30k.csv**

#### Période: Décembre 2025

# **1. Introduction et problématique**



La sécurité publique constitue un enjeu central pour les grandes villes, où les forces de l'ordre doivent faire face à une criminalité à la fois diffuse dans l'espace et variable dans le temps. Dans ce contexte, disposer d'outils d'analyse permettant de transformer les données de criminalité en décisions opérationnelles devient essentiel pour orienter les patrouilles, cibler les actions de prévention et affecter efficacement les effectifs disponibles.

La base de données Crime30.csv, qui recense les incidents criminels enregistrés dans la ville de Chicago sur une période donnée, offre un support riche pour ce travail. Chaque incident y est décrit par le type d'infraction, les circonstances (lieu, heure, description, contexte domestique), l'issue policière (arrestation ou non), ainsi que par des informations temporelles et spatiales fines (date, heure, quartier, coordonnées). Cette structure permet à la fois une lecture globale des niveaux de criminalité et une analyse plus ciblée des profils d'infractions selon le temps, l'espace et la réponse policière.

Dans ce projet, l'objectif n'est donc pas seulement de “décrire” la criminalité, mais de l'analyser dans une perspective d'aide à la décision : identifier les zones et créneaux horaires les plus exposés, repérer les types d'infractions les plus consommateurs de ressources, et proposer des critères concrets pour allouer de nouveaux agents sur le territoire.




Notre problématique consiste alors à identifier les profils spatio-temporels d'infractions et orienter de manière optimale l'allocation des effectifs policiers (en termes de lieux, de périodes et de compétences).

Afin de résoudre cette problématique, nous allons à travers cette étude, essayer de répondre aux questions suivantes:

* *Quels profils de criminalité urbaine observe-t-on dans la zone étudiée, en combinant l'espace (beats, districts, quartiers), le temps (heure, jour, saison, année), le type d'infraction, le contexte domestique et l'issue policière (arrestation ou non)?*

* *Quels facteurs (type d'infraction, localisation, créneau horaire, contexte domestique, durée de traitement) sont significativement associés à la probabilité d'arrestation et à la charge opérationnelle des services de police?*

* *Comment les types de criminalité et la pression opérationnelle évoluent-ils selon les quartiers et la temporalité (jours, saisons, années), et comment ces dynamiques peuvent-elles être traduites en priorités d'affectation pour les nouveaux agents?*

# **2. Exploration et préparation des données**

In [1]:
# Importattion des libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
import warnings
from matplotlib.ticker import MaxNLocator

# Chargement des données
data = pd.read_csv('crimes30k.csv')


In [2]:
data.head(5)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,11021273,JA351713,07/10/2017 09:50:00 AM,002XX N MICHIGAN AVE,0820,THEFT,$500 AND UNDER,COMMERCIAL / BUSINESS OFFICE,False,False,...,42.0,32.0,06,NaN,NaN,2017,07/18/2017 03:52:43 PM,NaN,NaN,NaN
1,10928464,JA242180,07/30/2012 11:05:00 AM,027XX W 84TH PL,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,NaN,False,False,...,18.0,70.0,11,NaN,NaN,2012,04/29/2017 03:49:38 PM,NaN,NaN,NaN
2,10122816,HY311916,12/10/2008 12:00:00 PM,085XX S ESCANABA AVE,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,RESIDENCE,False,False,...,10.0,46.0,11,NaN,NaN,2008,08/17/2015 03:03:40 PM,NaN,NaN,NaN
3,1448084,G177358,03/28/2001 10:00:00 PM,005XX E 33 ST,5001,OTHER OFFENSE,OTHER CRIME INVOLVING PROPERTY,PARKING LOT/GARAGE(NON.RESID.),False,False,...,NaN,NaN,26,NaN,NaN,2001,08/17/2015 03:03:40 PM,NaN,NaN,NaN
4,1793881,G617937,10/14/2001 03:50:00 PM,062XX N MC CORMICK RD,0460,BATTERY,SIMPLE,PARKING LOT/GARAGE(NON.RESID.),True,False,...,NaN,NaN,08B,NaN,NaN,2001,08/17/2015 03:03:40 PM,NaN,NaN,NaN
